In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter
from collections import OrderedDict
# from ipynb.fs.full.dev_notebook import stringToQiskitSingleGate

from generator import generateCliffordCircuit

In [2]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [3]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

In [4]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['S', 'X', 'Z', 'X', 'I'], ['I', 'I', 'I', 'CNOT_T', 'CNOT_C'], ['S', 'H', 'S', 'H', 'H'], ['I', 'I', 'CNOT_C', 'CNOT_T', 'I'], ['X', 'X', 'H', 'Z', 'X']]
     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ S ├─░───────░─┤ S ├─░───────░─┤ X ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_1: ┤ X ├─░───────░─┤ H ├─░───────░─┤ X ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_2: ┤ Z ├─░───────░─┤ S ├─░───■───░─┤ H ├
     ├───┤ ░ ┌───┐ ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_3: ┤ X ├─░─┤ X ├─░─┤ H ├─░─┤ X ├─░─┤ Z ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░ └───┘ ░ ├───┤
q_4: ┤ I ├─░───■───░─┤ H ├─░───────░─┤ X ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [5]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [6]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('ZXYZX')

In [29]:
def params_list(operation_set, pauli_set=["X","Y","Z","I"]):
    '''This function generates a dictionary of parameters for the given operation set.
    Inputs: operation_set - A list of operations.
            pauli_set - A list of pauli operators.
    Outputs: params - A dictionary of parameters, with each set to 0.'''

    params = OrderedDict()
    for operation in operation_set:
        if operation == "I":
            continue
        elif operation == "CNOT_C" or operation == "CNOT_T" or operation == "CX":
            for pauli1 in pauli_set:
                for pauli2 in pauli_set:
                    if pauli1 == "I" and pauli2 == "I":
                        continue
                    pauli = pauli1+pauli2
                    params[("CNOT", pauli)] = 0
        # elif operation == "CX":
        #     for pauli1 in pauli_set:
        #         for pauli2 in pauli_set:
        #             if pauli1 == "I" or pauli2 == "I":
        #                 continue
        #             pauli = pauli1+pauli2
        #             params[("CX", pauli)] = 0
        else:
            for pauli in pauli_set:
                if pauli == "I":
                    continue
                params[(operation, pauli)] = 0
    return params

print(params_list(singleGateSet+doubleGateSet))
print(len(list(params_list(singleGateSet+doubleGateSet).keys())))

OrderedDict([(('X', 'X'), 0), (('X', 'Y'), 0), (('X', 'Z'), 0), (('H', 'X'), 0), (('H', 'Y'), 0), (('H', 'Z'), 0), (('Z', 'X'), 0), (('Z', 'Y'), 0), (('Z', 'Z'), 0), (('S', 'X'), 0), (('S', 'Y'), 0), (('S', 'Z'), 0), (('CNOT', 'XX'), 0), (('CNOT', 'XY'), 0), (('CNOT', 'XZ'), 0), (('CNOT', 'XI'), 0), (('CNOT', 'YX'), 0), (('CNOT', 'YY'), 0), (('CNOT', 'YZ'), 0), (('CNOT', 'YI'), 0), (('CNOT', 'ZX'), 0), (('CNOT', 'ZY'), 0), (('CNOT', 'ZZ'), 0), (('CNOT', 'ZI'), 0), (('CNOT', 'IX'), 0), (('CNOT', 'IY'), 0), (('CNOT', 'IZ'), 0)])
27


In [9]:
def generate_first_layer(length, pauli_set=["X","Y","Z","I"]):
    first_layer = ""
    for ele in np.random.choice(pauli_set, length):
        first_layer += ele
    return first_layer.upper()

print(generate_first_layer(5))

IZZIY


In [ ]:
def pauli_hist(circuit, pauli_initial, params_list=params_list(singleGateSet)):
    '''This function computes the histogram of pauli operators after the circuit has been applied.
    Inputs: circuit - A Qiskit Circuit, Clifford, or Gate object.
            pauli_initial - A Qiskit Pauli object.
            params_list - A dictionary of parameters for the circuit.
    Outputs: pauli_counts - A dictionary of pauli operators and their counts.'''

    pauli = pauli_initial

    for op_instructions in circuit.data:
        if op_instructions.operation.name.upper() == "ID" or op_instructions.operation.name.upper() == "I" or op_instructions.operation.name.upper() == "BARRIER":
            print("skipped ", op_instructions.operation.name.upper(), " operation")
            break
        elif op_instructions.operation.name.upper() == "CX" or op_instructions.operation.name.upper() == "CNOT":
            params_list[("CNOT_C", pauli.to_label()[0])] += 1
            params_list[("CNOT_T", pauli.to_label()[1])] += 1
            pauli = qk.quantum_info.Pauli(pauli.to_label()[1] + pauli.to_label()[0] + pauli.to_label()[2:])
        else:
            pauli = remove_negatives(pauli)
            params_list[(op_instructions.operation.name.upper(), pauli.to_label())] += 1
            pauli = Clifford_Permute(op_instructions.operation, pauli)

    return params_list

print(pauli_hist(qiskitCircuit, qk.quantum_info.Pauli('X')))

In [54]:
def generate_row_A(circuit, input_pauli, params_dict=params_list(singleGateSet+doubleGateSet)):
    '''This function generates a row of the circuit, which is a dictionary of pauli operators and their counts.'''
    # generate the random circuit
    print(random_circuit)

    if type(input_pauli) == str:
        input_pauli_str = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_str = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase

    # get the following information from circuit: 
    # 1) input pauli layer ex. 'XIZII', with an initial list of the paulis at each register that can be overwritten.
    # 2) empty dictionary of pauli operators and their counts ex. 
    # {('X', 'X'): 0, ('X', 'I'): 0, ('Z', 'Z'): 0, ('Z', 'I'): 0, ('I', 'X'): 0, ('I', 'I'): 0}
    # 3) all operations in the circuit ex. ['X', 'H', 'Z', 'I', 'S', 'CNOT_C', 'CNOT_T']
    # input_pauli_str= generate_first_layer(random_circuit.num_qubits)
    pauli_str = input_pauli_str
    # input_pauli = [char for char in input_pauli_layer]
    curr_row_dict = params_dict.copy()
    all_operations = curr_row_dict.keys()
    i_count = 0
    # iterate each gate in each layer of the circuit 
    for layer in split_circuit_by_barrier(random_circuit):
        for gate in layer:
            # get the register of the current gate
            register = random_circuit.find_bit(gate.qubits[0]).index

            # get the gate's operator and the input pauli
            input_gate = gate.operation.name.upper()
            cur_pauli = pauli_str[register]
            # if cur_pauli == 'I':
            #     continue
            
            # print statement for debugging
            # print("current operation is: ", register, input_gate, cur_pauli)

            if (input_gate, cur_pauli) in all_operations:
                curr_row_dict[(input_gate, cur_pauli)] += 1
            elif input_gate == 'CX' or input_gate == 'CNOT':
                cur_pauli = pauli_str[layer.data[0][1][0].index] + pauli_str[layer.data[0][1][1].index]
                if cur_pauli == "II":
                    continue 
                curr_row_dict[('CNOT', cur_pauli)] += 1
                continue
            elif input_gate == 'ID' or input_gate == 'I' or input_gate == 'BARRIER':
                i_count += 1
                print("skipped ", input_gate, " operation")
                continue
            else:
                # continue
                print("Bad", input_gate, cur_pauli)

        # use predefined function to update the pauli layer
        pauli = Clifford_Permute(layer, qk.quantum_info.Pauli(pauli_str))
        pauli_str = ''.join(char for char in pauli.to_label() if char.isalpha())

    
    # print statement for debugging
    print("X total = ", curr_row_dict[("X", "X")]+curr_row_dict[("X", "Y")]+curr_row_dict[("X", "Z")])
    # print("Y total = ", curr_row_dict[("Y", "X")]+curr_row_dict[("Y", "Y")]+curr_row_dict[("Y", "Z")])
    print("Z total = ", curr_row_dict[("Z", "X")]+curr_row_dict[("Z", "Y")]+curr_row_dict[("Z", "Z")])
    print("H total = ", curr_row_dict[("H", "X")]+curr_row_dict[("H", "Y")]+curr_row_dict[("H", "Z")])
    print("S total = ", curr_row_dict[("S", "X")]+curr_row_dict[("S", "Y")]+curr_row_dict[("S", "Z")])
    print("CNOT total = ", curr_row_dict[("CNOT", "XX")]+curr_row_dict[("CNOT", "XY")]+curr_row_dict[("CNOT", "XZ")]+curr_row_dict[("CNOT", "YX")]+curr_row_dict[("CNOT", "YY")]+curr_row_dict[("CNOT", "YZ")]+curr_row_dict[("CNOT", "ZX")]+curr_row_dict[("CNOT", "ZY")]+curr_row_dict[("CNOT", "ZZ")])


    print("total # of operations done = ",  sum(curr_row_dict.values()))
    print("total should be = ", 17-i_count)
    print(curr_row_dict)
    return curr_row_dict.values()

random_circuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
input_pauli_str = generate_first_layer(random_circuit.num_qubits)
print(input_pauli_str)
generate_row_A(random_circuit, input_pauli_str)

IZXZZ
     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ I ├─░───────░─┤ S ├─░───────░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_1: ┤ I ├─░───────░─┤ I ├─░───────░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ ┌───┐ ░ ├───┤
q_2: ┤ S ├─░───■───░─┤ Z ├─░─┤ X ├─░─┤ Z ├
     ├───┤ ░   │   ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_3: ┤ Z ├─░───┼───░─┤ Z ├─░───■───░─┤ X ├
     ├───┤ ░ ┌─┴─┐ ░ ├───┤ ░       ░ ├───┤
q_4: ┤ H ├─░─┤ X ├─░─┤ Z ├─░───────░─┤ Z ├
     └───┘ ░ └───┘ ░ └───┘ ░       ░ └───┘
skipped  ID  operation
skipped  ID  operation
X total =  1
Z total =  6
H total =  2
S total =  2
CNOT total =  2
total # of operations done =  13
total should be =  15
OrderedDict([(('X', 'X'), 0), (('X', 'Y'), 0), (('X', 'Z'), 1), (('H', 'X'), 1), (('H', 'Y'), 0), (('H', 'Z'), 1), (('Z', 'X'), 0), (('Z', 'Y'), 2), (('Z', 'Z'), 4), (('S', 'X'), 2), (('S', 'Y'), 0), (('S', 'Z'), 0), (('CNOT', 'XX'), 0), (('CNOT', 'XY'), 0), (('CNOT', 'XZ'), 0), (('CNOT', 'XI'), 0), (('CNOT', 'YX'), 0), (('CNOT', 'YY'), 0), (('CNOT',

/var/folders/9r/sw7rn4810sjddctl8q8nlcy80000gn/T/ipykernel_67549/1318717272.py:40: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  cur_pauli = pauli_str[layer.data[0][1][0].index] + pauli_str[layer.data[0][1][1].index]


odict_values([0, 0, 1, 1, 0, 1, 0, 2, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [261]:
def generate_A(width, depth, params_dict=params_list(singleGateSet)):
    '''This function generates the A matrix by randomly generating each row, checking if it increases the rank and then continuing onwards
    Inputs: numrow - The number of rows in the A matrix.
            numcol - The number of columns in the A matrix.
    Outputs: A - The A matrix.'''

    A = []
    counter = 0
    while np.linalg.matrix_rank(A) < len(list(params_dict.keys())):
        random_circuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
        input_pauli_str = generate_first_layer(random_circuit.num_qubits)
        new_row = list(generate_row_A(random_circuit, input_pauli_str, params_dict))
        A_star = A.copy()
        A_star.append(new_row)
        if np.linalg.matrix_rank(np.array(A_star)) > np.linalg.matrix_rank(np.array(A)):
            print("to get to rank: ", np.linalg.matrix_rank(A), ", it took us ", counter, "tries")
            A = A_star
        else:
            counter += 1
            # print(new_row)
        if counter > 100:
            break
            # depth += 1
            # counter = 0
            # print("depth = ", depth)
    return np.array(A)

In [264]:
A = generate_A(width = 5, depth = 50)
print(A)
print(np.linalg.matrix_rank(A))

to get to rank:  0 , it took us  0 tries
to get to rank:  1 , it took us  0 tries
to get to rank:  2 , it took us  0 tries
to get to rank:  3 , it took us  0 tries
to get to rank:  4 , it took us  0 tries
to get to rank:  5 , it took us  0 tries
to get to rank:  6 , it took us  0 tries
to get to rank:  7 , it took us  0 tries
to get to rank:  8 , it took us  0 tries
to get to rank:  9 , it took us  0 tries
to get to rank:  10 , it took us  0 tries
[[0 1 1 1 1 0 1 1 0 0 2 0]
 [1 0 1 0 0 2 1 1 0 0 2 1]
 [3 0 0 1 1 1 1 1 0 2 1 1]
 [2 1 0 1 2 0 1 1 0 2 2 0]
 [0 1 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 1 1 0 1 1 0]
 [0 0 0 0 0 1 1 1 0 0 1 1]
 [0 0 1 0 0 2 0 0 2 0 0 3]
 [1 1 1 1 0 0 0 0 2 1 0 2]
 [1 0 1 3 0 0 1 1 0 3 0 1]
 [0 2 0 1 1 1 1 1 0 0 2 1]]
11


In [237]:
for i in range(10):
    random_circuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
    input_pauli_str = generate_first_layer(random_circuit.num_qubits)
    print(list(generate_row_A(random_circuit, input_pauli_str)))

[1, 1, 3, 0, 2, 1, 0, 1, 1, 0, 0, 0]
[1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 1, 2]
[0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 1]
[0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 0]
[0, 1, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0]
[2, 0, 0, 2, 1, 1, 0, 1, 1, 1, 0, 1]
[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 1, 1, 0, 3, 0, 2, 1, 0, 0, 2, 0]
[0, 1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1]
[3, 0, 0, 1, 1, 0, 1, 1, 0, 1, 2, 0]
